# TAVOLE STATISTICHE

In [1]:
import sqlite3, pandas as pd, requests, os, sys, sqlalchemy, duckdb
from io import BytesIO
import json
from pyjstat import pyjstat

In [ ]:
conn = sqlite3.connect("D:/files/Bankit.sqlite")
from sqlalchemy import create_engine
sqlite = create_engine('sqlite:///D:/files/Bankit.sqlite')

def carica_dati_in_sql(tabella):
    dtypes = {
        "DESINV": sqlalchemy.types.INTEGER(),
        "DURORI": sqlalchemy.types.INTEGER(),
        "TIPTASSO": sqlalchemy.types.INTEGER(),
        "VALORE": sqlalchemy.types.INTEGER(),
        "CLASSE_ACCORD":sqlalchemy.types.TEXT()
    }
    data.to_sql(tabella, sqlite, if_exists='replace', index=False, dtype=dtypes)
    return

file_path = 'D:\\DatiStatistici.xlsx'
# file_path = 'C:\\Users\\PVolterr\\Mediocredito Centrale S.p.A\\Studi e Governo Iniziative - Documenti\\dati statistici.xlsx'
from openpyxl import load_workbook
# os.chdir('C:\\Users\\PVolterr\\Mediocredito Centrale S.p.A\\Studi e Governo Iniziative - Documenti')
from sqlalchemy import create_engine

db_file = 'D:/files/Bankit.duckdb' # Nome del file del database
ddb = duckdb.connect(db_file) # Connessione al database DuckDB (crea il file se non esiste)

query = "SELECT * FROM `domain-stacoris-multicube`"
stacoris = pd.read_sql_query(query, conn)
query = "SELECT * FROM `domain-stafinra-multicube`"
stafinra = pd.read_sql_query(query, conn)
query = "SELECT * FROM `domain-stamen-multicube`"
stamen = pd.read_sql_query(query, conn)

nuts1 = ['IT','ITC','ITC1','ITC2','ITC3','ITC4','ITH','ITH3','ITH4','ITH5','ITHBI12','ITI','ITI1','ITI3','ITI4','ITF','ITF1','ITF2','ITF3','ITF4','ITF5','ITF6','ITG','ITG1','ITG2']

### TDB10255 - sostituito nel 2017 da TFR10255

In [18]:
tabella = 'TFR10255'
file = f'https://a2a.bancaditalia.it/infostat/dataservices/export/IT/CSV/DATA/CUBE/BANKITALIA/DIFF/{tabella}'
result = requests.get(file)
date_column = ['DATA_OSS']
data = pd.read_csv(BytesIO(result.content),compression='zip', header=0, sep=';', quotechar='"', encoding='utf-8',dtype={'ENTE_SEGN':'str', 'FENEC':'str', 'VALORE':'Int32','LOC_SPORT':'Int32'},parse_dates=date_column, dayfirst=False)
data['DATA_OSS'] = pd.to_datetime(data['DATA_OSS'])
# data = data[data['DATA_OSS'] == data['DATA_OSS'].max()]
# data['DATA_OSS'] = data['DATA_OSS'].dt.date
# data = data[data['LOC_CTP'].isin(nuts1)]
data = pd.merge(data, stafinra, how = 'left', left_on='ENTE_SEGN', right_on='Elemento').drop(columns=['STATUS','FENEC','Dominio','Elemento']).rename(columns={'Descrizione': 'segnalante'})
data = pd.merge(data, stafinra, how = 'left', left_on='LOC_CTP', right_on='Elemento').drop(columns=['Dominio', 'Elemento']).rename(columns={'Descrizione': 'area'})
data = pd.merge(data, stafinra, how = 'left', left_on='SET_CTP', right_on='Elemento').drop(columns=['Dominio', 'Elemento']).rename(columns={'Descrizione': 'target'})
data = pd.merge(data, stafinra, how = 'left', left_on='ATECO_CTP', right_on='Elemento').drop(columns=['Dominio', 'Elemento']).rename(columns={'Descrizione': 'ATECO'})
# data = data[['DATA_OSS', 'LOC_CTP', 'area', 'SET_CTP','target','VALORE']]
data.sample()

,ATECO_CTP,DATA_OSS,ENTE_SEGN,LOC_CTP,SET_CTP,VALORE,segnalante,area,target,ATECO
19795,1000063,2020-03-31,1070001,ITF6,SBI25,25503,Banche e Cassa depositi e prestiti,Calabria,Società non finanziarie e famiglie produttrici,"Carta, articoli di carta e prodotti della stampa"


In [19]:
carica_dati_in_sql(tabella)
data.shape

(58625, 10)

In [21]:
ddb.execute(f"DROP TABLE IF EXISTS {tabella}")
ddb.execute(f"CREATE TABLE IF NOT EXISTS {tabella} AS SELECT * FROM data LIMIT 0")  # Crea una tabella vuota con la struttura del DataFrame
ddb.execute(f"INSERT INTO {tabella} SELECT * FROM data")

In [ ]:
sheet_to_update = tabella
book = load_workbook(file_path)
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    data.to_excel(writer, sheet_name=sheet_to_update, index=False)

### TDB10266 DEPOSITS | Distribution by customer location (geographical area) and branch of economic activity

In [7]:
tabella = 'TDB10266'
file = f'https://a2a.bancaditalia.it/infostat/dataservices/export/IT/CSV/DATA/CUBE/BANKITALIA/DIFF/{tabella}'
result = requests.get(file)
date_column = ['DATA_OSS']
data = pd.read_csv(BytesIO(result.content),compression='zip', header=0, sep=';', quotechar='"', encoding='utf-8',dtype={'ENTE_SEGN':'str', 'FENEC':'str', 'VALORE':'Int32','LOC_SPORT':'Int32'},parse_dates=date_column, dayfirst=False)
data['DATA_OSS'] = pd.to_datetime(data['DATA_OSS'])
data

,DATA_OSS,DIVISA1,DURORI,ENTE_SEGN,FENEC,LOC_CTP,RAMATECO,RESIDENZA1,SET_CTP,VALORE,STATUS
0,2008-09-30,1000,9,1100010,1041810,IT,51,IT,SBI25,8747852,NaN
1,2008-09-30,1000,9,1100010,1041810,IT,52,IT,SBI25,8035032,NaN
2,2008-09-30,1000,9,1100010,1041810,IT,53,IT,SBI25,2132191,NaN
3,2008-09-30,1000,9,1100010,1041810,IT,54,IT,SBI25,2827730,NaN
4,2008-09-30,1000,9,1100010,1041810,IT,55,IT,SBI25,3117736,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6187,1998-03-31,1000,9,1100010,1041810,ITI,70,IT,SBI25,118544,NaN
6188,1998-03-31,1000,9,1100010,1041810,ITI,71,IT,SBI25,281756,NaN
6189,1998-03-31,1000,9,1100010,1041810,ITI,72,IT,SBI25,83151,NaN
6190,1998-03-31,1000,9,1100010,1041810,ITI,73,IT,SBI25,3836204,NaN


In [8]:
data['DATA_OSS'].max()

Timestamp('2008-09-30 00:00:00')

In [ ]:

# data = data[data['DATA_OSS'] == data['DATA_OSS'].max()]
# data['DATA_OSS'] = data['DATA_OSS'].dt.date
# data = data[data['LOC_CTP'].isin(nuts1)]
data = pd.merge(data, stamen, how = 'left', left_on='ENTE_SEGN', right_on='Elemento').drop(columns=['ENTE_SEGN','STATUS','FENEC','index','Dominio','Elemento']).rename(columns={'Descrizione': 'segnalante'})
data = pd.merge(data, stamen, how = 'left', left_on='LOC_CTP', right_on='Elemento').drop(columns=['index','Dominio', 'Elemento']).rename(columns={'Descrizione': 'area'})
data = pd.merge(data, stamen, how = 'left', left_on='SET_CTP', right_on='Elemento').drop(columns=['index','Dominio', 'Elemento']).rename(columns={'Descrizione': 'target'})
# data = data[['DATA_OSS', 'LOC_CTP', 'area', 'SET_CTP','target','VALORE']]
dtypes = {"DIVISA1": sqlalchemy.types.INTEGER(), "DURORI": sqlalchemy.types.INTEGER(),"LOC_SPORT": sqlalchemy.types.INTEGER(),
         "VALORE": sqlalchemy.types.INTEGER()}
data.to_sql('TDB10266', sqlite, if_exists='replace', dtype=dtypes,index=False)



In [ ]:
sheet_to_update = tabella
book = load_workbook(file_path)
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    data.to_excel(writer, sheet_name=sheet_to_update, index=False)

### TDB10290

In [6]:
tabella = 'TDB10290'
file = f'https://a2a.bancaditalia.it/infostat/dataservices/export/IT/CSV/DATA/CUBE/BANKITALIA/DIFF/{tabella}'
result = requests.get(file)
date_column = ['DATA_OSS']
data = pd.read_csv(BytesIO(result.content),compression='zip', header=0, sep=';', quotechar='"', encoding='utf-8',dtype={'ENTE_SEGN':'str', 'FENEC':'str', 'VALORE':'Int32','LOC_SPORT':'Int32'},parse_dates=date_column, dayfirst=False)
data['DATA_OSS'] = pd.to_datetime(data['DATA_OSS'])
data = data[data['DATA_OSS'] == data['DATA_OSS'].max()]
data['DATA_OSS'] = data['DATA_OSS'].dt.date
data = data[data['LOC_CTP'].isin(nuts1)]
data = pd.merge(data, stamen, how = 'left', left_on='ENTE_SEGN', right_on='Elemento').drop(columns=['ENTE_SEGN','STATUS','FENEC','index','Dominio','Elemento']).rename(columns={'Descrizione': 'segnalante'})
data = pd.merge(data, stamen, how = 'left', left_on='LOC_CTP', right_on='Elemento').drop(columns=['index','Dominio', 'Elemento']).rename(columns={'Descrizione': 'area'})
data = pd.merge(data, stamen, how = 'left', left_on='SET_CTP', right_on='Elemento').drop(columns=['index','Dominio', 'Elemento']).rename(columns={'Descrizione': 'target'})
data = data[['DATA_OSS', 'LOC_CTP', 'area', 'SET_CTP','target','VALORE']]
sheet_to_update = tabella
book = load_workbook(file_path)
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    data.to_excel(writer, sheet_name=sheet_to_update, index=False)

In [7]:
tabella = 'TDB10295'
file = f'https://a2a.bancaditalia.it/infostat/dataservices/export/IT/CSV/DATA/CUBE/BANKITALIA/DIFF/{tabella}'
result = requests.get(file)
date_column = ['DATA_OSS']
data = pd.read_csv(BytesIO(result.content),compression='zip', header=0, sep=';', quotechar='"', encoding='utf-8',dtype={'ENTE_SEGN':'str', 'FENEC':'str', 'VALORE':'Int32','LOC_SPORT':'Int32'},parse_dates=date_column, dayfirst=False)
data['DATA_OSS'] = pd.to_datetime(data['DATA_OSS'])
data = data[data['DATA_OSS'] == data['DATA_OSS'].max()]
data['DATA_OSS'] = data['DATA_OSS'].dt.date
data = data[data['LOC_CTP'].isin(nuts1)]
data = pd.merge(data, stamen, how = 'left', left_on='ENTE_SEGN', right_on='Elemento').drop(columns=['ENTE_SEGN','STATUS','FENEC','index','Dominio','Elemento']).rename(columns={'Descrizione': 'segnalante'})
data = pd.merge(data, stamen, how = 'left', left_on='LOC_CTP', right_on='Elemento').drop(columns=['index','Dominio', 'Elemento']).rename(columns={'Descrizione': 'area'})
data = pd.merge(data, stamen, how = 'left', left_on='SET_CTP', right_on='Elemento').drop(columns=['index','Dominio', 'Elemento']).rename(columns={'Descrizione': 'target'})
data = data[['DATA_OSS', 'LOC_CTP', 'area', 'SET_CTP','target','VALORE']]
sheet_to_update = tabella
book = load_workbook(file_path)
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    data.to_excel(writer, sheet_name=sheet_to_update, index=False)

In [ ]:
tabella = 'TDB20290'
file = f'https://a2a.bancaditalia.it/infostat/dataservices/export/IT/CSV/DATA/CUBE/BANKITALIA/DIFF/{tabella}'
result = requests.get(file)
date_column = ['DATA_OSS']
data = pd.read_csv(BytesIO(result.content),compression='zip', header=0, sep=';', quotechar='"', encoding='utf-8',dtype={'ENTE_SEGN':'str', 'FENEC':'str', 'VALORE':'Int32','LOC_SPORT':'Int32'},parse_dates=date_column, dayfirst=False)
data['DATA_OSS'] = pd.to_datetime(data['DATA_OSS'])
#data = data[data['DATA_OSS'] == data['DATA_OSS'].max()]
data['DATA_OSS'] = data['DATA_OSS'].dt.date
#data = data[data['LOC_CTP'].isin(nuts1)]
data = pd.merge(data, stamen, how = 'left', left_on='ENTE_SEGN', right_on='Elemento').drop(columns=['ENTE_SEGN','STATUS','FENEC','index','Dominio','Elemento']).rename(columns={'Descrizione': 'segnalante'})
data = pd.merge(data, stamen, how = 'left', left_on='LOC_CTP', right_on='Elemento').drop(columns=['index','Dominio', 'Elemento']).rename(columns={'Descrizione': 'area'})
data = pd.merge(data, stamen, how = 'left', left_on='SET_CTP', right_on='Elemento').drop(columns=['index','Dominio', 'Elemento']).rename(columns={'Descrizione': 'target'})
data = data[['DATA_OSS', 'LOC_CTP', 'area', 'SET_CTP','target','VALORE']]
sheet_to_update = tabella
book = load_workbook(file_path)
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    data.to_excel(writer, sheet_name=sheet_to_update, index=False)

In [19]:
tabella = 'TFR20231'
file = f'https://a2a.bancaditalia.it/infostat/dataservices/export/IT/CSV/DATA/CUBE/BANKITALIA/DIFF/{tabella}'
result = requests.get(file)
date_column = ['DATA_OSS']
data = pd.read_csv(BytesIO(result.content),compression='zip', header=0, sep=';', quotechar='"', decimal=',', encoding='utf-8',dtype={'ENTE_SEGN':'str', 'FENEC':'str', 'VALORE':'float64','LOC_SPORT':'Int32'},parse_dates=date_column, dayfirst=False)
data['DATA_OSS'] = pd.to_datetime(data['DATA_OSS'])
data = data[data['DATA_OSS'] == data['DATA_OSS'].max()]
data['DATA_OSS'] = data['DATA_OSS'].dt.date
data = data[data['LOC_CTP'].isin(nuts1)]
data = pd.merge(data, stamen, how = 'left', left_on='ENTE_SEGN', right_on='Elemento').drop(columns=['ENTE_SEGN','STATUS','FENEC','index','Dominio','Elemento']).rename(columns={'Descrizione': 'segnalante'})
data = pd.merge(data, stamen, how = 'left', left_on='LOC_CTP', right_on='Elemento').drop(columns=['index','Dominio', 'Elemento']).rename(columns={'Descrizione': 'area'})
data = pd.merge(data, stamen, how = 'left', left_on='SET_CTP', right_on='Elemento').drop(columns=['index','Dominio', 'Elemento']).rename(columns={'Descrizione': 'target'})
#data = pd.merge(data, stamen, how = 'left', left_on='ATECO_CTP', right_on='Elemento').drop(columns=['index','Dominio', 'Elemento']).rename(columns={'Descrizione': 'ATECO'})
data = data[['DATA_OSS', 'LOC_CTP', 'area', 'SET_CTP','target','VALORE']] # 'ATECO_CTP', 'ATECO',
sheet_to_update = tabella
book = load_workbook(file_path)
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    data.to_excel(writer, sheet_name=sheet_to_update, index=False)

In [20]:
tabella = 'TRI30603'
file = f'https://a2a.bancaditalia.it/infostat/dataservices/export/IT/CSV/DATA/CUBE/BANKITALIA/DIFF/{tabella}'
result = requests.get(file)
date_column = ['DATA_OSS']
data = pd.read_csv(BytesIO(result.content),compression='zip', header=0, sep=';', quotechar='"', decimal=',', encoding='utf-8',dtype={'ENTE_SEGN':'str', 'FENEC':'str', 'VALORE':'float64','LOC_SPORT':'Int32'},parse_dates=date_column, dayfirst=False)
data['DATA_OSS'] = pd.to_datetime(data['DATA_OSS'])
data = data[data['DATA_OSS'] == data['DATA_OSS'].max()]
data['DATA_OSS'] = data['DATA_OSS'].dt.date
data = data[data['SEDELEG_SOGG'].isin(nuts1)]
data = pd.merge(data, stamen, how = 'left', left_on='ENTE_SEGN', right_on='Elemento').drop(columns=['ENTE_SEGN','STATUS','FENEC','index','Dominio','Elemento']).rename(columns={'Descrizione': 'segnalante'})
data = pd.merge(data, stamen, how = 'left', left_on='SEDELEG_SOGG', right_on='Elemento').drop(columns=['index','Dominio', 'Elemento']).rename(columns={'Descrizione': 'area'})
data = pd.merge(data, stamen, how = 'left', left_on='SET_CTP', right_on='Elemento').drop(columns=['index','Dominio', 'Elemento']).rename(columns={'Descrizione': 'target'})
#data = pd.merge(data, stamen, how = 'left', left_on='ATECO_CTP', right_on='Elemento').drop(columns=['index','Dominio', 'Elemento']).rename(columns={'Descrizione': 'ATECO'})
data = data[['DATA_OSS', 'SEDELEG_SOGG', 'area', 'SET_CTP','target','VALORE']] # 'ATECO_CTP', 'ATECO',
sheet_to_update = tabella
book = load_workbook(file_path)
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    data.to_excel(writer, sheet_name=sheet_to_update, index=False)

In [21]:
# TRI30146 Prestiti (escluse sofferenze) - per regione della clientela e classe di grandezza del fido globale accordato 
tabella = 'TRI30146'
file = f'https://a2a.bancaditalia.it/infostat/dataservices/export/IT/CSV/DATA/CUBE/BANKITALIA/DIFF/{tabella}'
result = requests.get(file)
date_column = ['DATA_OSS']
data = pd.read_csv(BytesIO(result.content),compression='zip', header=0, sep=';', quotechar='"', decimal=',', encoding='utf-8',dtype={'ENTE_SEGN':'str', 'FENEC':'str', 'VALORE':'float64','LOC_SPORT':'Int32'},parse_dates=date_column, dayfirst=False)
data['DATA_OSS'] = pd.to_datetime(data['DATA_OSS'])
data = data[data['DATA_OSS'] == data['DATA_OSS'].max()]
data['DATA_OSS'] = data['DATA_OSS'].dt.date
data = data[data['SEDELEG_SOGG'].isin(nuts1)]
sheet_to_update = tabella
book = load_workbook(file_path)
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    data.to_excel(writer, sheet_name=sheet_to_update, index=False)

## Istat

In [1]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

In [5]:
# Valori pro capite
dataset = '93_1227_DF_DCCN_TNA1_6'
data_url = f'https://esploradati.istat.it/SDMXWS/rest/data/IT1,{dataset},1.0/A........./ALL/?detail=full&startPeriod=2014-01-01&endPeriod=2023-12-31&dimensionAtObservation=TIME_PERIOD'
response = requests.get(data_url)
root = ET.fromstring(response.content)
# 🔎 Namespace XML per SDMX aggiornato
ns = {"message": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message","generic": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic"}
data = []
for series in root.findall(".//generic:Series", namespaces=ns):
    series_key = series.find("generic:SeriesKey", namespaces=ns)
    attributes = {value.attrib["id"]: value.attrib["value"] for value in series_key.findall("generic:Value", namespaces=ns)}
    for obs in series.findall("generic:Obs", namespaces=ns):
        time_period = obs.find("generic:ObsDimension", namespaces=ns).attrib["value"]
        obs_value = obs.find("generic:ObsValue", namespaces=ns).attrib["value"]
        data.append({**attributes, "TIME_PERIOD": time_period, "OBS_VALUE": float(obs_value)})
df = pd.DataFrame(data)
# Valori pro capite
df1 = df.query('VALUATION == "V" and TIME_PERIOD == "2022" and DATA_TYPE_AGGR in ["B1G_B_W2_S1_R_POP","B1GQ_B_W2_S1_R_POP","B1G_B_W2_S1"]'
).replace("B1GQ_B_W2_S1_R_POP", "PIL/abitante").replace("B1G_B_W2_S1_R_POP", "Val.Aggiunto/abitante")[['EDITION','TIME_PERIOD', 'REF_AREA', 'DATA_TYPE_AGGR', 'OBS_VALUE']]
ValProCapite = pd.pivot_table(df1, index = 'REF_AREA', columns='DATA_TYPE_AGGR', values = 'OBS_VALUE')
ValProCapite.to_excel(writer, sheet_name='ValProCapite', startrow=3, startcol=2)

sheet_to_update = tabella
book = load_workbook(file_path)
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df.to_excel(writer, sheet_name=sheet_to_update, index=False)

NameError: name 'writer' is not defined

## CCIAA Macerata

In [6]:
url = "https://opendata.marche.camcom.it/data/Stock-Imprese-Attive-Italia-Natura-Giuridica.json"
response = requests.get(url)
json_stat_data = response.json()
dataset = pyjstat.Dataset(json_stat_data)
df = dataset.write('dataframe')

c:\Users\PVolterr\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyjstat\pyjstat.py:246: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dim_index = pd.DataFrame(list(zip([dim_label['id'][0]], [0])),


In [7]:
sheet_to_update = 'NatGiur'
book = load_workbook(file_path)
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df.to_excel(writer, sheet_name=sheet_to_update, index=False)

## backup

In [ ]:
import sqlite3
import pandas as pd
import requests
from io import BytesIO
from openpyxl import load_workbook

# Connessione SQLite
conn = sqlite3.connect("D:/files/Bankit.sqlite")

# Lista delle tabelle da elaborare
tabelle = [
    'TDB10224', 'TDB10226', 'TDB10290', 'TDB10295', 'TFR10255', 'TFR20231', 'TRI30603', 'TRI30146'
]

# Caricamento dei dati di riferimento
stacoris = pd.read_sql_query("SELECT * FROM `domain-stacoris-multicube`", conn)
stafinra = pd.read_sql_query("SELECT * FROM `domain-stafinra-multicube`", conn)
stamen = pd.read_sql_query("SELECT * FROM `domain-stamen-multicube`", conn)

# Chiudere la connessione al database
conn.close()

# Definizione NUTS1
nuts1 = ['IT', 'ITC', 'ITC1', 'ITC2', 'ITC3', 'ITC4', 'ITH', 'ITH3', 'ITH4', 'ITH5', 'ITHBI12', 'ITI', 'ITI1', 'ITI3', 'ITI4', 'ITF', 'ITF1', 'ITF2', 'ITF3', 'ITF4', 'ITF5', 'ITF6', 'ITG', 'ITG1', 'ITG2']

# Percorso file di output
file_path = 'D:/butta.xlsx'

# Funzione per elaborare e salvare ogni tabella
def process_table(tabella):
    print(f"Elaborazione tabella: {tabella}")
    url = f'https://a2a.bancaditalia.it/infostat/dataservices/export/IT/CSV/DATA/CUBE/BANKITALIA/DIFF/{tabella}'
    result = requests.get(url)
    
    data = pd.read_csv(BytesIO(result.content), compression='zip', header=0, sep=';', quotechar='"',
                        encoding='utf-8', dtype={'ENTE_SEGN': 'str', 'FENEC': 'str', 'VALORE': 'float64', 'LOC_SPORT': 'Int32'},
                        parse_dates=['DATA_OSS'], dayfirst=False)
    
    # Filtro sull'ultima data
    data['DATA_OSS'] = pd.to_datetime(data['DATA_OSS']).dt.date
    data = data[data['DATA_OSS'] == data['DATA_OSS'].max()]
    
    # Filtro sui codici NUTS1
    if 'LOC_CTP' in data.columns:
        data = data[data['LOC_CTP'].isin(nuts1)]
    elif 'SEDELEG_SOGG' in data.columns:
        data = data[data['SEDELEG_SOGG'].isin(nuts1)]
    
    # Merge con stamen per ottenere descrizioni
    colonne_merge = ['ENTE_SEGN', 'LOC_CTP', 'SET_CTP', 'ATECO_CTP']
    for col in colonne_merge:
        if col in data.columns:
            data = data.merge(stamen, how='left', left_on=col, right_on='Elemento').drop(columns=['index', 'Dominio', 'Elemento']).rename(columns={'Descrizione': col.lower()})
    
    # Selezione colonne finali
    colonne_finali = [col for col in ['DATA_OSS', 'LOC_CTP', 'segnalante', 'area', 'ATECO_CTP', 'ATECO', 'SET_CTP', 'target', 'VALORE'] if col in data.columns]
    data = data[colonne_finali]
    
    # Salvataggio su Excel
    book = load_workbook(file_path)
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        data.to_excel(writer, sheet_name=tabella, index=False)
    print(f"Tabella {tabella} elaborata e salvata con successo.")

# Elaborazione di tutte le tabelle
for tabella in tabelle:
    process_table(tabella)
